# Project - Google Analytics Customer Revenue Preprocessing

## Presenting the initial data: 

<b>Data Fields: </b>

<b>fullVisitorIdv</b> - A unique identifier for each user of the Google Merchandise Store. <br>
<b>channelGrouping</b> - The channel via which the user came to the Store.<br>
<b>date</b> - The date on which the user visited the Store.<br>
<b>device </b>- The specifications for the device used to access the Store.<br>
<b>geoNetwork</b> - This section contains information about the geography of the user.<br>
<b>sessionId</b> - A unique identifier for this visit to the store.<br>
<b>socialEngagementType</b> - Engagement type, either "Socially Engaged" or "Not Socially Engaged".<br>
<b>totals</b> - This section contains aggregate values across the session.<br>
<b>trafficSource</b> - This section contains information about the Traffic Source from which the session originated.<br>
<b>visitId</b> - An identifier for this session. This is part of the value usually stored as the _utmb cookie. This is only unique to the user. For a completely unique ID, you should use a combination of fullVisitorId and visitId.<br>
<b>visitNumber</b> - The session number for this user. If this is the first session, then this is set to 1.<br>
<b>visitStartTime</b> - The timestamp (expressed as POSIX time).<br>

# Objectives: 

The main objectives of this project are :

* Load the data so everything is in tabular format (some columns contain JSON so it you will need to find ways to separate those into independent columns)
* Identify the variables that need special processing (removing or infering missing values, removing columns that don't contain useful information)
* Run visualizations to better understand the data

tip : the ```pd.read_csv``` function has some very useful arguments that will help you read the data properly. Use the ```converters``` argument along with the ```json.loads``` function in order to read the json columns , ```dtype``` enables you to set the type of specific columns, you may also use the ```skiprows``` argument in order to load a fraction of the dataset for faster subsequent processing.

## Importing necessary libraries

In [1]:
# Necessary librarys
import os # it's a operational system library, to set some informations
import random # random is to generate random values

import pandas as pd # to manipulate data frames 
import numpy as np # to work with matrix

import matplotlib.pyplot as plt # to graphics plot
import seaborn as sns # a good library to graphic plots

# Importing librarys to use on interactive graphs
from plotly.offline import init_notebook_mode, iplot, plot 
import plotly.graph_objs as go 

import json # to convert json in df
from pandas.io.json import json_normalize # to normalize the json file

# to set a style to all graphs
plt.style.use('fivethirtyeight')
init_notebook_mode(connected=True)

## Some columns are in Json format so it will be necessary to handle them.

# Importing the datasets

In [6]:
%%time 
# %%time is used to calculate the timing of code chunk execution #

# We will import the data using the name and extension that will be concatenated with dir_path
df = pd.read_csv("s3://full-stack-bigdata-datasets/Machine Learning Supervisé/projects/preprocessing_linear_models/Google_dataset_sample.csv", low_memory=False) 
# The same to test dataset
#df_test = json_read("test.csv") 

CPU times: user 2.4 s, sys: 423 ms, total: 2.83 s
Wall time: 5.03 s


In [7]:
# This command shows the first 5 rows of our dataset
pd.set_option('display.max_columns', 500)

df.head()

channelGrouping      date        fullVisitorId  \
0  Organic Search  20160902  4763447161404445595   
1  Organic Search  20160902  1905672039242460897   
2  Organic Search  20160902  3696906537737368442   
3  Organic Search  20160902  8794587387581803040   
4  Organic Search  20160902  1438836965936298791   

                        sessionId  socialEngagementType     visitId  \
0  4763447161404445595_1472881213  Not Socially Engaged  1472881213   
1  1905672039242460897_1472817241  Not Socially Engaged  1472817241   
2  3696906537737368442_1472856874  Not Socially Engaged  1472856874   
3  8794587387581803040_1472816048  Not Socially Engaged  1472816048   
4  1438836965936298791_1472833322  Not Socially Engaged  1472833322   

   visitNumber  visitStartTime     device.browser  \
0            1      1472881213         UC Browser   
1            1      1472817241             Chrome   
2            1      1472856874             Chrome   
3            1      1472816048  Internet Explorer   
4            1      1472833322  Internet Explorer   

           device.browserVersion             device.browserSize  \
0  not available in demo dataset  not available in demo dataset   
1  not available in demo dataset  not available in demo dataset   
2  not available in demo dataset  not available in demo dataset   
3  not available in demo dataset  not available in demo dataset   
4  not available in demo dataset  not available in demo dataset   

  device.operatingSystem  device.operatingSystemVersion  device.isMobile  \
0                  Linux  not available in demo dataset            False   
1                Windows  not available in demo dataset            False   
2                Windows  not available in demo dataset            False   
3                Windows  not available in demo dataset            False   
4                Windows  not available in demo dataset            False   

     device.mobileDeviceBranding       device.mobileDeviceModel  \
0  not available in demo dataset  not available in demo dataset   
1  not available in demo dataset  not available in demo dataset   
2  not available in demo dataset  not available in demo dataset   
3  not available in demo dataset  not available in demo dataset   
4  not available in demo dataset  not available in demo dataset   

      device.mobileInputSelector        device.mobileDeviceInfo  \
0  not available in demo dataset  not available in demo dataset   
1  not available in demo dataset  not available in demo dataset   
2  not available in demo dataset  not available in demo dataset   
3  not available in demo dataset  not available in demo dataset   
4  not available in demo dataset  not available in demo dataset   

  device.mobileDeviceMarketingName            device.flashVersion  \
0    not available in demo dataset  not available in demo dataset   
1    not available in demo dataset  not available in demo dataset   
2    not available in demo dataset  not available in demo dataset   
3    not available in demo dataset  not available in demo dataset   
4    not available in demo dataset  not available in demo dataset   

                 device.language            device.screenColors  \
0  not available in demo dataset  not available in demo dataset   
1  not available in demo dataset  not available in demo dataset   
2  not available in demo dataset  not available in demo dataset   
3  not available in demo dataset  not available in demo dataset   
4  not available in demo dataset  not available in demo dataset   

         device.screenResolution device.deviceCategory geoNetwork.continent  \
0  not available in demo dataset               desktop                 Asia   
1  not available in demo dataset               desktop                 Asia   
2  not available in demo dataset               desktop             Americas   
3  not available in demo dataset               desktop               Europe   
4  not available in demo dataset               desktop          

## Knowing the missing values

## Creating the function to handle with date 

In [8]:
# library of datetime
from datetime import datetime

# This function is to extract date features
def date_process(df):
    df["date"] = pd.to_datetime(df["date"], format="%Y%m%d") # seting the column as pandas datetime
    df["_weekday"] = df['date'].dt.weekday #extracting week day
    df["_day"] = df['date'].dt.day # extracting day
    df["_month"] = df['date'].dt.month # extracting day
    df["_year"] = df['date'].dt.year # extracting day
    df['_visitHour'] = (df['visitStartTime'].apply(lambda x: str(datetime.fromtimestamp(x).hour))).astype(int)
    
    return df #returning the df after the transformations

In [9]:
df_train = date_process(df) #calling the function that we created above

## Deal with missing values

In [10]:
def NumericalColumns(df):    # fillna numeric feature
    df['totals.pageviews'].fillna(1, inplace=True) #filling NA's with 1
    df['totals.newVisits'].fillna(0, inplace=True) #filling NA's with 0
    df['totals.bounces'].fillna(0, inplace=True)   #filling NA's with 0
    df['trafficSource.isTrueDirect'].fillna(False, inplace=True) # filling boolean with False
    df['trafficSource.adwordsClickInfo.isVideoAd'].fillna(True, inplace=True) # filling boolean with True
    df["totals.transactionRevenue"] = df["totals.transactionRevenue"].fillna(0.0).astype(float) #filling NA with zero
    df['totals.pageviews'] = df['totals.pageviews'].astype(int) # setting numerical column as integer
    df['totals.newVisits'] = df['totals.newVisits'].astype(int) # setting numerical column as integer
    df['totals.bounces'] = df['totals.bounces'].astype(int)  # setting numerical column as integer
    df["totals.hits"] = df["totals.hits"].astype(float) # setting numerical to float
    df['totals.visits'] = df['totals.visits'].astype(int) # seting as int

    return df #return the transformed dataframe

In [14]:
from sklearn import preprocessing

def Normalizing(df):
    # Use MinMaxScaler to normalize the column
    df["totals.hits"] =  (df['totals.hits'] - min(df['totals.hits'])) / (max(df['totals.hits'])  - min(df['totals.hits']))
    # normalizing the transaction Revenue
    df['totals.transactionRevenue'] = df_train['totals.transactionRevenue'].apply(lambda x: np.log10(x+1))
    # return the modified df
    return df 

In [15]:
# call the function to transform the numerical columns
df_train = NumericalColumns(df_train)

# Call the function that will normalize some features
df_train = Normalizing(df_train)

In [16]:
df_train.head()

channelGrouping       date        fullVisitorId  \
0  Organic Search 2016-09-02  4763447161404445595   
1  Organic Search 2016-09-02  1905672039242460897   
2  Organic Search 2016-09-02  3696906537737368442   
3  Organic Search 2016-09-02  8794587387581803040   
4  Organic Search 2016-09-02  1438836965936298791   

                        sessionId  socialEngagementType     visitId  \
0  4763447161404445595_1472881213  Not Socially Engaged  1472881213   
1  1905672039242460897_1472817241  Not Socially Engaged  1472817241   
2  3696906537737368442_1472856874  Not Socially Engaged  1472856874   
3  8794587387581803040_1472816048  Not Socially Engaged  1472816048   
4  1438836965936298791_1472833322  Not Socially Engaged  1472833322   

   visitNumber  visitStartTime     device.browser  \
0            1      1472881213         UC Browser   
1            1      1472817241             Chrome   
2            1      1472856874             Chrome   
3            1      1472816048  Internet Explorer   
4            1      1472833322  Internet Explorer   

           device.browserVersion             device.browserSize  \
0  not available in demo dataset  not available in demo dataset   
1  not available in demo dataset  not available in demo dataset   
2  not available in demo dataset  not available in demo dataset   
3  not available in demo dataset  not available in demo dataset   
4  not available in demo dataset  not available in demo dataset   

  device.operatingSystem  device.operatingSystemVersion  device.isMobile  \
0                  Linux  not available in demo dataset            False   
1                Windows  not available in demo dataset            False   
2                Windows  not available in demo dataset            False   
3                Windows  not available in demo dataset            False   
4                Windows  not available in demo dataset            False   

     device.mobileDeviceBranding       device.mobileDeviceModel  \
0  not available in demo dataset  not available in demo dataset   
1  not available in demo dataset  not available in demo dataset   
2  not available in demo dataset  not available in demo dataset   
3  not available in demo dataset  not available in demo dataset   
4  not available in demo dataset  not available in demo dataset   

      device.mobileInputSelector        device.mobileDeviceInfo  \
0  not available in demo dataset  not available in demo dataset   
1  not available in demo dataset  not available in demo dataset   
2  not available in demo dataset  not available in demo dataset   
3  not available in demo dataset  not available in demo dataset   
4  not available in demo dataset  not available in demo dataset   

  device.mobileDeviceMarketingName            device.flashVersion  \
0    not available in demo dataset  not available in demo dataset   
1    not available in demo dataset  not available in demo dataset   
2    not available in demo dataset  not available in demo dataset   
3    not available in demo dataset  not available in demo dataset   
4    not available in demo dataset  not available in demo dataset   

                 device.language            device.screenColors  \
0  not available in demo dataset  not available in demo dataset   
1  not available in demo dataset  not available in demo dataset   
2  not available in demo dataset  not available in demo dataset   
3  not available in demo dataset  not available in demo dataset   
4  not available in demo dataset  not available in demo dataset   

         device.screenResolution device.deviceCategory geoNetwork.continent  \
0  not available in demo dataset               desktop                 Asia   
1  not available in demo dataset               desktop                 Asia   
2  not available in demo dataset               desktop             Americas   
3  not available in demo dataset               desktop               Europe   
4  not available in demo dataset               desktop    

In [17]:
# let's remove the date, sessionId, visitId, visitNumber, visitStartTime, geoNetwork.region, geoNetwork.metro, geoNetwork.city, geoNetwork.networkDomain
# let's transform all the _weekday, _day, _month, _year, _visitHour to string so we can dummyfy them
df_clean = df_train.drop(["date", "sessionId", "visitId", "visitNumber", "visitStartTime", "geoNetwork.region", "geoNetwork.metro", "geoNetwork.city", "geoNetwork.networkDomain",
"trafficSource.source",	"trafficSource.medium", "trafficSource.isTrueDirect",	"trafficSource.adwordsClickInfo.isVideoAd",	"trafficSource.campaignCode", "geoNetwork.continent",	"geoNetwork.subContinent", "_day"], axis = 1)

transform_to_string = ["_weekday", "_month", "_year", "_visitHour"]
for col in transform_to_string:
    df_clean[col] = df_clean[col].astype(str)

df_id = df_clean["fullVisitorId"]
df_no_id = df_clean.drop(["fullVisitorId"], axis=1)

object_variables = df_no_id.select_dtypes(include = "object")
non_object_variables = df_no_id.select_dtypes(exclude = "object")

category_to_replace = []
for col in object_variables :
    value_proportion_table = object_variables[col].value_counts()/len(object_variables)
    columns_to_replace = [col for col in value_proportion_table.keys() if value_proportion_table[col]>0.01]
    category_to_replace.append(columns_to_replace)

for i, col in enumerate(object_variables.columns) :
    object_variables[col] = np.where(object_variables[col].isin(category_to_replace[i]),object_variables[col], "others")

df_no_id = pd.concat([object_variables,non_object_variables], axis=1)

# dummyfy the variables
df_clean = pd.get_dummies(df_no_id, drop_first=True)
df_clean["fullVisitorId"] = df_id
df_clean.head()

<ipython-input-17-ba891292603c>:23: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



device.isMobile  totals.visits  totals.hits  totals.pageviews  \
0            False              1          0.0                 1   
1            False              1          0.0                 1   
2            False              1          0.0                 1   
3            False              1          0.0                 1   
4            False              1          0.0                 1   

   totals.bounces  totals.newVisits  totals.transactionRevenue  \
0               1                 1                        0.0   
1               1                 1                        0.0   
2               1                 1                        0.0   
3               1                 1                        0.0   
4               1                 1                        0.0   

   trafficSource.adwordsClickInfo.page  channelGrouping_Direct  \
0                                  NaN                       0   
1                                  NaN                       0   
2                                  NaN                       0   
3                                  NaN                       0   
4                                  NaN                       0   

   channelGrouping_Organic Search  channelGrouping_Paid Search  \
0                               1                            0   
1                               1                            0   
2                               1                            0   
3                               1                            0   
4                               1                            0   

   channelGrouping_Referral  channelGrouping_Social  channelGrouping_others  \
0                         0                       0                       0   
1                         0                       0                       0   
2                         0                       0                       0   
3                         0                       0                       0   
4                         0                       0                       0   

   device.browser_Edge  device.browser_Firefox  \
0                    0                       0   
1                    0                       0   
2                    0                       0   
3                    0                       0   
4                    0                       0   

   device.browser_Internet Explorer  device.browser_Safari  \
0                                 0                      0   
1                                 0                      0   
2                                 0                      0   
3                                 1                      0   
4                                 1                      0   

   device.browser_others  device.operatingSystem_Chrome OS  \
0                      1                                 0   
1                      0                                 0   
2                      0                                 0   
3                      0                                 0   
4                      0                                 0   

   device.operatingSystem_Linux  device.operatingSystem_Macintosh  \
0                             1                                 0   
1                             0                                 0   
2                             0                                 0   
3                             0                                 0   
4                             0                                 0   

   device.operatingSystem_Windows  device.operatingSystem_iOS  \
0                               0                           0   
1                               1                           0   
2                               1                           0   
3                               1                           0   
4                               1                           0   

   device.operatingSystem_others  device.deviceCategory_mobile  \
0        

In [18]:
df_clean.shape

(126410, 106)

In [19]:
# let's group and aggregate data thanks to the fullVisitorId
df_agg = df_clean.groupby("fullVisitorId").sum()
df_agg.head()

device.isMobile  totals.visits  totals.hits  \
fullVisitorId                                                      
0000114156543135683                0              1     0.000000   
0000450371054833295                0              1     0.000000   
0000538867824729259                0              1     0.000000   
0000639845445148063                0              1     0.014028   
000072202462202136                 1              1     0.000000   

                     totals.pageviews  totals.bounces  totals.newVisits  \
fullVisitorId                                                             
0000114156543135683                 1               1                 1   
0000450371054833295                 1               1                 1   
0000538867824729259                 1               1                 1   
0000639845445148063                 7               0                 1   
000072202462202136                  1               1                 1   

                     totals.transactionRevenue  \
fullVisitorId                                    
0000114156543135683                        0.0   
0000450371054833295                        0.0   
0000538867824729259                        0.0   
0000639845445148063                        0.0   
000072202462202136                         0.0   

                     trafficSource.adwordsClickInfo.page  \
fullVisitorId                                              
0000114156543135683                                  0.0   
0000450371054833295                                  0.0   
0000538867824729259                                  0.0   
0000639845445148063                                  0.0   
000072202462202136                                   0.0   

                     channelGrouping_Direct  channelGrouping_Organic Search  \
fullVisitorId                                                                 
0000114156543135683                       0                               0   
0000450371054833295                       0                               0   
0000538867824729259                       1                               0   
0000639845445148063                       1                               0   
000072202462202136                        0                               1   

                     channelGrouping_Paid Search  channelGrouping_Referral  \
fullVisitorId                                                                
0000114156543135683                            0                         0   
0000450371054833295                            0                         1   
0000538867824729259                            0                         0   
0000639845445148063                            0                         0   
000072202462202136                             0                         0   

                     channelGrouping_Social  channelGrouping_others  \
fullVisitorId                                                         
0000114156543135683                       1                       0   
0000450371054833295                       0                       0   
0000538867824729259                       0                       0   
0000639845445148063                       0                       0   
000072202462202136                        0                       0   

                     device.browser_Edge  device.browser_Firefox  \
fullVisitorId                                                      
0000114156543135683                    0                       0   
0000450371054833295                    0                       0   
0000538867824729259                    0                       0   
0000639845445148063                    0                       0   
000072202462202136                     0                       0   

                     device.browser_Internet Explorer  device.browser_Safari  \
fullVisitorId                                                                  
0000114156

In [20]:
# Let's separate target and training variables

y = df_agg["totals.transactionRevenue"]
X = df_agg.drop(["totals.transactionRevenue"], axis=1)

In [21]:
# split the data between train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3)

In [22]:
# normalize the data
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

X_train = pd.DataFrame(sc.fit_transform(X_train), columns = X_train.columns, index= X_train.index)
X_test = pd.DataFrame(sc.fit_transform(X_test), columns = X_test.columns, index= X_test.index)

X_train.head()

device.isMobile  totals.visits  totals.hits  \
fullVisitorId                                                      
9427870688042783967         0.867244      -0.195738    -0.144457   
9294875467542463120        -0.439296      -0.195738    -0.201055   
9456658598495244920        -0.439296      -0.195738    -0.257653   
6891084756410794026        -0.439296       0.542776     0.025336   
2387066031914314449        -0.439296      -0.195738    -0.257653   

                     totals.pageviews  totals.bounces  totals.newVisits  \
fullVisitorId                                                             
9427870688042783967         -0.132253       -0.673633          0.110470   
9294875467542463120         -0.202585       -0.673633          0.110470   
9456658598495244920         -0.272917        0.398372          0.110470   
6891084756410794026          0.149074        0.398372          0.110470   
2387066031914314449         -0.272917        0.398372         -7.136843   

                     trafficSource.adwordsClickInfo.page  \
fullVisitorId                                              
9427870688042783967                            -0.089394   
9294875467542463120                            -0.089394   
9456658598495244920                            -0.089394   
6891084756410794026                            -0.089394   
2387066031914314449                            -0.089394   

                     channelGrouping_Direct  channelGrouping_Organic Search  \
fullVisitorId                                                                 
9427870688042783967               -0.276217                        0.517198   
9294875467542463120               -0.276217                       -0.582488   
9456658598495244920               -0.276217                       -0.582488   
6891084756410794026               -0.276217                        1.616884   
2387066031914314449               -0.276217                        0.517198   

                     channelGrouping_Paid Search  channelGrouping_Referral  \
fullVisitorId                                                                
9427870688042783967                    -0.099283                 -0.225012   
9294875467542463120                    -0.099283                 -0.225012   
9456658598495244920                    -0.099283                 -0.225012   
6891084756410794026                    -0.099283                 -0.225012   
2387066031914314449                    -0.099283                 -0.225012   

                     channelGrouping_Social  channelGrouping_others  \
fullVisitorId                                                         
9427870688042783967               -0.427510               -0.052703   
9294875467542463120                0.913306               -0.052703   
9456658598495244920                0.913306               -0.052703   
6891084756410794026               -0.427510               -0.052703   
2387066031914314449               -0.427510               -0.052703   

                     device.browser_Edge  device.browser_Firefox  \
fullVisitorId                                                      
9427870688042783967            -0.049148               -0.154935   
9294875467542463120            -0.049148               -0.154935   
9456658598495244920            -0.049148               -0.154935   
6891084756410794026            -0.049148               -0.154935   
2387066031914314449            -0.049148                2.827051   

                     device.browser_Internet Explorer  device.browser_Safari  \
fullVisitorId                                                                  
9427870688042783967                         -0.073584               1.167185   
9294875467542463120                         -0.073584               1.167185   
9456658598495244920                         -0.073584               1.167185   
6891084756410794026                          5.599213              -0.401913   
2387066031914314449                      

In [27]:
# let's train a model
from statsmodels.api import OLS

X_train["constant"] = 1
X_test["constant"] = 1
model = OLS(y_train, X_train)

In [28]:
model_fit = model.fit()

In [29]:
model_fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                OLS Regression Results                               
=====================================================================================
Dep. Variable:     totals.transactionRevenue   R-squared:                       0.420
Model:                                   OLS   Adj. R-squared:                  0.419
Method:                        Least Squares   F-statistic:                     490.9
Date:                       Fri, 18 Dec 2020   Prob (F-statistic):               0.00
Time:                               16:38:33   Log-Likelihood:                 48513.
No. Observations:                      70000   AIC:                        -9.682e+04
Df Residuals:                          69896   BIC:                        -9.587e+04
Df Model:                                103                                         
Covariance Type:                   nonrobust                                         
=============================================================================================================================
                                                                coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------------------
device.isMobile                                               0.0212      0.017      1.233      0.217      -0.012       0.055
totals.visits                                              6.262e+10   4.72e+10      1.327      0.184   -2.99e+10    1.55e+11
totals.hits                                                   0.0121      0.003      3.785      0.000       0.006       0.018
totals.pageviews                                              0.0810      0.003     23.897      0.000       0.074       0.088
totals.bounces                                                0.0078      0.001      9.820      0.000       0.006       0.009
totals.newVisits                                              0.0020      0.000      4.316      0.000       0.001       0.003
trafficSource.adwordsClickInfo.page                           0.0030      0.007      0.407      0.684      -0.012       0.018
channelGrouping_Direct                                     -3.36e+10   2.53e+10     -1.327      0.184   -8.32e+10     1.6e+10
channelGrouping_Organic Search                            -4.205e+10   3.17e+10     -1.327      0.184   -1.04e+11    2.01e+10
channelGrouping_Paid Search                               -1.652e+10   1.24e+10     -1.327      0.184   -4.09e+10    7.88e+09
channelGrouping_Referral                                  -3.045e+10   2.29e+10     -1.327      0.184   -7.54e+10    1.45e+10
channelGrouping_Social                                    -3.449e+10    2.6e+10     -1.327      0.184   -8.54e+10    1.64e+10
channelGrouping_others                                    -7.872e+09   5.93e+09     -1.327      0.184   -1.95e+10    3.75e+09
device.browser_Edge                                          -0.0024      0.001     -4.197      0.000      -0.004      -0.001
device.browser_Firefox                                       -0.0002      0.001     -0.343      0.732      -0.001       0.001
device.browser_Internet Explorer                              0.0132      0.001     24.305      0.000       0.012       0.014
device.browser_Safari                                         0.0027      0.001      3.666      0.000       0.001       0.004
device.browser_others                                         0.0017      0.001      3.323      0.001       0.001       0.003
device.operatingSystem_Chrome OS                              0.0053      0.003      2.004      0.045       0.000       0.010
device.operatingSystem_Linux                                 -0.0048      0.002     -2.565      0.010      -0.009      -0.001
device.operatingSystem_Macintosh                              0.0004      0.006      0.068      0.946      -

In [30]:
# remove strongly correlated variables
corr = X.corr()
high_corr = corr > 0.95
high_corr_list = [(i,j) for i in range(corr.shape[0]) for j in range(corr.shape[0]) if i != j and high_corr.iloc[i,j]]
high_corr_list

[(1, 61),
 (1, 62),
 (2, 3),
 (3, 2),
 (47, 50),
 (50, 47),
 (61, 1),
 (61, 62),
 (62, 1),
 (62, 61)]

In [31]:
no_keep = []
unique_couples = []
for couple in high_corr_list :
    if (couple[1],couple[0]) not in unique_couples:
        unique_couples.append(couple)
        no_keep.append(couple[1])

X_train = X_train.drop(X_train.columns[no_keep], axis=1)
X_test = X_test.drop(X_test.columns[no_keep], axis=1)

In [32]:
print(X.columns[no_keep])

Index(['trafficSource.adwordsClickInfo.slot_others',
       'trafficSource.adwordsClickInfo.adNetworkType_others',
       'totals.pageviews', 'trafficSource.keyword_6qEhsCssdK0z36ri',
       'trafficSource.adwordsClickInfo.adNetworkType_others'],
      dtype='object')


In [33]:
model = OLS(y_train, X_train)
model_fit = model.fit()
model_fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                OLS Regression Results                               
=====================================================================================
Dep. Variable:     totals.transactionRevenue   R-squared:                       0.415
Model:                                   OLS   Adj. R-squared:                  0.414
Method:                        Least Squares   F-statistic:                     500.6
Date:                       Fri, 18 Dec 2020   Prob (F-statistic):               0.00
Time:                               16:38:55   Log-Likelihood:                 48218.
No. Observations:                      70000   AIC:                        -9.624e+04
Df Residuals:                          69900   BIC:                        -9.532e+04
Df Model:                                 99                                         
Covariance Type:                   nonrobust                                         
=============================================================================================================================
                                                                coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------------------
device.isMobile                                               0.0187      0.017      1.086      0.277      -0.015       0.053
totals.visits                                              1.881e+09   3.31e+10      0.057      0.955   -6.31e+10    6.68e+10
totals.hits                                                   0.0871      0.001    141.988      0.000       0.086       0.088
totals.bounces                                                0.0054      0.001      7.078      0.000       0.004       0.007
totals.newVisits                                              0.0022      0.000      4.702      0.000       0.001       0.003
trafficSource.adwordsClickInfo.page                           0.0220      0.004      5.184      0.000       0.014       0.030
channelGrouping_Direct                                     -1.01e+09   1.78e+10     -0.057      0.955   -3.59e+10    3.38e+10
channelGrouping_Organic Search                            -1.263e+09   2.22e+10     -0.057      0.955   -4.49e+10    4.23e+10
channelGrouping_Paid Search                               -4.962e+08   8.74e+09     -0.057      0.955   -1.76e+10    1.66e+10
channelGrouping_Referral                                   -9.15e+08   1.61e+10     -0.057      0.955   -3.25e+10    3.07e+10
channelGrouping_Social                                    -1.036e+09   1.82e+10     -0.057      0.955   -3.68e+10    3.47e+10
channelGrouping_others                                    -2.365e+08   4.16e+09     -0.057      0.955    -8.4e+09    7.93e+09
device.browser_Edge                                          -0.0022      0.001     -3.756      0.000      -0.003      -0.001
device.browser_Firefox                                       -0.0002      0.001     -0.348      0.728      -0.001       0.001
device.browser_Internet Explorer                              0.0134      0.001     24.540      0.000       0.012       0.014
device.browser_Safari                                         0.0028      0.001      3.765      0.000       0.001       0.004
device.browser_others                                         0.0017      0.001      3.194      0.001       0.001       0.003
device.operatingSystem_Chrome OS                              0.0037      0.003      1.405      0.160      -0.001       0.009
device.operatingSystem_Linux                                 -0.0059      0.002     -3.139      0.002      -0.010      -0.002
device.operatingSystem_Macintosh                             -0.0033      0.006     -0.542      0.588      -0.015       0.009
device.operatingSystem_Windows                               -0.0014      0.005     -0.268      0.789      -

In [38]:
# Now let's try lasso in order to remove bad predictors
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso

params = {'alpha' : [10**(-a) for a in range(10)]}
lasso = Lasso()
grid = GridSearchCV(lasso,param_grid=params, cv = 10, verbose=1)

grid.fit(X_train,y_train)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: 101.03862535795793, tolerance: 0.15473778397559532

/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: 75.8965298476287, tolerance: 0.1640723187765464

/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: 65.49735244335977, tolerance: 0.16499283553720895

/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning:

Objective did not converge. You m

/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: 473.15277707684425, tolerance: 0.16499283553720895

/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: 473.3925832013896, tolerance: 0.166130589789124

/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: 467.76812761675274, tolerance: 0.15925528758754312

/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: 463.7354200469688, 

GridSearchCV(cv=10, estimator=Lasso(),
             param_grid={'alpha': [1, 0.1, 0.01, 0.001, 0.0001, 1e-05, 1e-06,
                                   1e-07, 1e-08, 1e-09]},
             verbose=1)

In [39]:
print(grid.best_params_)

{'alpha': 0.001}


In [40]:
best_model = grid.best_estimator_
print("Score on the train set :", best_model.score(X_train,y_train))
print("Score on the test set :", best_model.score(X_test,y_test))

Score on the train set : 0.4106467480884891
Score on the test set : 0.17259610335638587


In [41]:
print("columns that have been removed with lasso : ", X_train.columns[best_model.coef_==0])

columns that have been removed with lasso :  Index(['totals.visits', 'totals.bounces',
       'trafficSource.adwordsClickInfo.page', 'channelGrouping_Organic Search',
       'device.browser_Firefox', 'device.browser_others',
       'device.operatingSystem_Macintosh', 'device.operatingSystem_Windows',
       'device.operatingSystem_iOS', 'device.deviceCategory_mobile',
       'geoNetwork.country_Brazil', 'geoNetwork.country_Germany',
       'geoNetwork.country_India', 'geoNetwork.country_Italy',
       'geoNetwork.country_Mexico', 'geoNetwork.country_Netherlands',
       'geoNetwork.country_Philippines', 'geoNetwork.country_Poland',
       'geoNetwork.country_Russia', 'geoNetwork.country_Turkey',
       'trafficSource.campaign_AW - Dynamic Search Ads Whole Site',
       'trafficSource.keyword_others', 'trafficSource.referralPath_/yt/about/',
       'trafficSource.referralPath_/yt/about/th/', '_weekday_1', '_weekday_2',
       '_weekday_3', '_weekday_4', '_month_12', '_month_3', '_month_

In [42]:
print("columns that have been kept with lasso : ", X_train.columns[best_model.coef_!=0])

columns that have been kept with lasso :  Index(['device.isMobile', 'totals.hits', 'totals.newVisits',
       'channelGrouping_Direct', 'channelGrouping_Paid Search',
       'channelGrouping_Referral', 'channelGrouping_Social',
       'channelGrouping_others', 'device.browser_Edge',
       'device.browser_Internet Explorer', 'device.browser_Safari',
       'device.operatingSystem_Chrome OS', 'device.operatingSystem_Linux',
       'device.operatingSystem_others', 'device.deviceCategory_tablet',
       'geoNetwork.country_Canada', 'geoNetwork.country_France',
       'geoNetwork.country_Indonesia', 'geoNetwork.country_Japan',
       'geoNetwork.country_Spain', 'geoNetwork.country_Taiwan',
       'geoNetwork.country_Thailand', 'geoNetwork.country_United Kingdom',
       'geoNetwork.country_United States', 'geoNetwork.country_Vietnam',
       'geoNetwork.country_others', 'trafficSource.campaign_Data Share Promo',
       'trafficSource.campaign_others',
       'trafficSource.referralPath_/an

In [43]:
# try with ridge
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge

params = {'alpha':np.arange(0,1000,10)} # determine the range of parameters to try
ridge = Ridge() # create an instance of the model

grid = GridSearchCV(ridge, params, cv=10, verbose = 1)
grid_fit = grid.fit(X_train, y_train)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:   39.2s finished


In [44]:
print(grid.best_params_)

{'alpha': 990}


In [45]:
best_model = grid.best_estimator_
print("Score on the train set :", best_model.score(X_train,y_train))
print("Score on the test set :", best_model.score(X_test,y_test))

Score on the train set : 0.41456803686697974
Score on the test set : 0.16324611210817042
